In [ ]:
import torch
import numpy as np
from sbi_particle_physics.objects.model import Model
from sbi_particle_physics.objects.normalizer import Normalizer
from sbi_particle_physics.managers.plotter import Plotter
from sbi_particle_physics.managers.backup import Backup
from sbi_particle_physics.config import DATA_DIR

In [ ]:
max_files = 10
device = "cpu"

In [ ]:
files = Backup.detect_files(DATA_DIR / "data_2")[:max_files]
raw_data, raw_parameters, _ = Backup.load_data(files, device)
n_samples = raw_data.shape[0]
n_points = raw_data.shape[1]

In [ ]:
print("Raw data")
print(raw_data.shape)
print(raw_data[0,:10])
unique_points, counts = torch.unique(
    raw_data[0],
    dim=0,
    return_counts=True
)
print("#unique points", unique_points.shape[0])

print("\nRaw parameters")
print(raw_parameters.shape)
print(raw_parameters[:10])

In [ ]:
normalizer = Normalizer.create_normalizer(device, raw_data)
data = normalizer.normalize_data(raw_data)
parameters = normalizer.normalize_parameters(raw_parameters)

In [ ]:
Plotter.plot_a_sample(data[0], parameters[0])

In [ ]:
# Analyse statistique : moyennes et écarts-types pour chaque observable
# Sélectionner quelques échantillons avec différentes valeurs de C_9

# Trier par valeur de C_9
sorted_indices = torch.argsort(raw_parameters.squeeze())
n_compare = 5  # Nombre d'échantillons à comparer

# Sélectionner des échantillons espacés (min, 25%, 50%, 75%, max)
indices_to_compare = sorted_indices[torch.linspace(0, len(sorted_indices)-1, n_compare).long()]

print("=" * 80)
print("COMPARAISON DES STATISTIQUES POUR DIFFÉRENTES VALEURS DE C_9")
print("=" * 80)

observable_names = ["q²", "cos(θ_l)", "cos(θ_d)", "φ"]

for idx in indices_to_compare:
    sample = raw_data[idx]
    param = raw_parameters[idx]
    
    print(f"\n{'─' * 80}")
    print(f"C_9 = {param.item():.4f}")
    print(f"{'─' * 80}")
    print(f"{'Observable':<15} {'Mean':<15} {'Std':<15} {'Min':<15} {'Max':<15}")
    print(f"{'─' * 80}")
    
    for i, obs_name in enumerate(observable_names):
        obs_data = sample[:, i]
        print(f"{obs_name:<15} {obs_data.mean():.4f}{'':<10} "
              f"{obs_data.std():.4f}{'':<10} "
              f"{obs_data.min():.4f}{'':<10} "
              f"{obs_data.max():.4f}{'':<10}")

print(f"\n{'=' * 80}")
print("RÉSUMÉ : Vérification de la variabilité entre échantillons")
print(f"{'=' * 80}")

# Calculer les moyennes pour tous les échantillons
all_means = raw_data.mean(dim=1)  # [n_samples, 4]

for i, obs_name in enumerate(observable_names):
    means_for_obs = all_means[:, i]
    print(f"\n{obs_name}:")
    print(f"  - Moyenne des moyennes : {means_for_obs.mean():.4f}")
    print(f"  - Écart-type des moyennes : {means_for_obs.std():.4f}")
    print(f"  - Min des moyennes : {means_for_obs.min():.4f}")
    print(f"  - Max des moyennes : {means_for_obs.max():.4f}")
    print(f"  => Variabilité = {means_for_obs.std():.4f} (devrait être > 0 si C_9 a un effet)")

In [ ]:
# Visualisation : comparer les distributions pour différentes valeurs de C_9
print("\nGénération des graphiques de comparaison...")
Plotter.compare_distributions(raw_data, raw_parameters, n_samples_to_plot=5)

In [ ]:
# Test de corrélation : vérifier si C_9 influence les distributions
# Calculer la corrélation entre C_9 et les moyennes de chaque observable

print("\n" + "=" * 80)
print("CORRÉLATION ENTRE C_9 ET LES OBSERVABLES")
print("=" * 80)

# Calculer les moyennes et écarts-types pour tous les échantillons
all_means = raw_data.mean(dim=1)  # [n_samples, 4]
all_stds = raw_data.std(dim=1)    # [n_samples, 4]

c9_values = raw_parameters.squeeze().cpu().numpy()

print("\nCorrélation entre C_9 et la MOYENNE de chaque observable:")
print(f"{'Observable':<15} {'Corrélation':<15} {'Interprétation':<30}")
print("─" * 80)

for i, obs_name in enumerate(observable_names):
    means = all_means[:, i].cpu().numpy()
    correlation = np.corrcoef(c9_values, means)[0, 1]
    
    if abs(correlation) > 0.5:
        interpretation = "FORTE dépendance"
    elif abs(correlation) > 0.2:
        interpretation = "Dépendance modérée"
    else:
        interpretation = "Faible dépendance"
    
    print(f"{obs_name:<15} {correlation:>+.4f}{'':<10} {interpretation:<30}")

print("\nCorrélation entre C_9 et l'ÉCART-TYPE de chaque observable:")
print(f"{'Observable':<15} {'Corrélation':<15} {'Interprétation':<30}")
print("─" * 80)

for i, obs_name in enumerate(observable_names):
    stds = all_stds[:, i].cpu().numpy()
    correlation = np.corrcoef(c9_values, stds)[0, 1]
    
    if abs(correlation) > 0.5:
        interpretation = "FORTE dépendance"
    elif abs(correlation) > 0.2:
        interpretation = "Dépendance modérée"
    else:
        interpretation = "Faible dépendance"
    
    print(f"{obs_name:<15} {correlation:>+.4f}{'':<10} {interpretation:<30}")

print("\n" + "=" * 80)
print("CONCLUSION:")
print("Si les corrélations sont proches de 0, C_9 n'a PAS d'effet sur les distributions.")
print("Si les corrélations sont significatives (> 0.2 en valeur absolue), C_9 INFLUENCE les distributions.")
print("=" * 80)